# Test effect of kmer size and base pairs used for training

In this notebook we will test the effect of kmer size used to produce images and of amount of data in each image in validation accuracy after training.

In all cases, we will use Label Smoothing, Cut Mix, and a pretrained xresnet50. We will include all samples sequenced for a given amount of data, regardless of the sequencing quality. Three randomly chosen samples will be used for validation and the remainder for training. Training will be done in 15 epochs, and the batch size will be chosen so that that are approximately 10 batches per epoch (up to a maximum size of 64). For each combination of kmer-size and amount of data, we will do 5 replicates with a different training/validation set.

In [1]:
import pandas as pd
from pathlib import Path
from fastai.data.all import DataBlock, ColSplitter, ColReader
from fastai.vision.all import ImageBlock, CategoryBlock, cnn_learner, xresnet50
from fastai.metrics import accuracy
from fastai.learner import load_learner
from fastai.torch_core import set_seed
from torch.cuda import get_device_name
from fastai.callback.mixup import CutMix
from fastai.losses import LabelSmoothingCrossEntropyFlat
from math import log
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

get_device_name(0)

'Tesla V100-PCIE-32GB'

Function: select training and validation set given kmer size and amount of data (as a list).
We will only consider samples that have yielded at least 200M bp

In [2]:
def get_training_set(kmer_size, bp_training):
    file_path = [x.absolute() for x in (Path('images_' + str(kmer_size))).ls() if x.suffix == '.png']
    taxon = [x.name.split('__')[0] for x in file_path]
    sample = [x.name.split('__')[-1].split('_')[0] for x in file_path]
    n_bp = [int(x.name.split('_')[-1].split('.')[0].replace('K','000')) for x in file_path]

    df = pd.DataFrame({'taxon': taxon,
                  'sample': sample,
                  'n_bp': n_bp,
                  'path': file_path
                 })
    
    included_samples = df.loc[df['n_bp'] == 200000000]['sample']
    df = df.loc[df['sample'].isin(included_samples)]

    valid = (df[['taxon','sample']].
             drop_duplicates().
             groupby('taxon').
             apply(lambda x: x.sample(3, replace=False)).
             reset_index(drop=True).
             assign(is_valid = True).
             merge(df, on = ['taxon','sample'])
        )

    train = (df.loc[~df['sample'].isin(valid['sample'])].
             assign(is_valid = False)
            )
    train = train.loc[train['n_bp'].isin(bp_training)]

    train_valid = pd.concat([valid, train]).reset_index(drop = True)
    return train_valid                          
                                    

Function: create a learner and fit model, setting batch size according to number of training images

In [3]:
def train_cnn(df, model = xresnet50, epochs = 30, normalize = True, pretrained = True, callbacks = CutMix, loss_fn = LabelSmoothingCrossEntropyFlat()):
    #find a batch size that is a power of 2 and splits the dataset in about 10 batches
    batch_size = min(2**round(log(df[~df['is_valid']].shape[0]/10,2)), 64) 
    
    #start data block
    dbl = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter = ColSplitter(),
                       get_x = ColReader('path'),
                       get_y = ColReader('taxon'),
                       item_tfms = None,
                       batch_tfms = None
                      )
    
    #create data loaders with calculated batch size
    dls = dbl.dataloaders(df, bs = batch_size)
    
    #create learner and train for 15 epochs
    learn = cnn_learner(dls, 
                    model, 
                    metrics = accuracy, 
                    normalize = normalize,
                    pretrained = pretrained,
                    cbs = callbacks,
                    loss_func = loss_fn
                   ).to_fp16()
    
    #to speed up training, we will skip validation
    valid = learn.dls.valid
    learn.dls.valid = []
    
    with learn.no_logging():
        learn.fine_tune(epochs = epochs, freeze_epochs = 0, base_lr = 1e-3)
        
    #now we can put the validation dataloader back    
    learn.dls.valid = valid
    
    return(learn)
    

Function: get overall accuracy given a model, a table and amount of basepairs

In [4]:
def get_accuracy(learner, df, bp_test):
    test = df[(df['is_valid'] == True) & df['n_bp'].isin(bp_test)]
    tdl = learner.dls.test_dl(test, with_labels = True)
    return learner.validate(dl = tdl)

Now that we defined functions, let's test do a for loop to do the test for all combinations of kmer length, bp_training, bp_valid

In [5]:
set_seed(1773541)

with open('kmerSize_VS_bp.txt','w') as outfile:
    for replicate in range(10):        
        for kmer_len in range(5,10):
            clear_output()
            print('Training using all bp amounts for kmer size', kmer_len)
            print('Replicate', replicate)
            all_bp_tr = sorted([mult * 10**power for mult in [1,2,5] for power in range(5,9) 
                                if mult * 10**power <= 200000000 and mult * 10**power >= 500000])

            #training with all bp amounts
            df = get_training_set(kmer_len, all_bp_tr)
            learn = train_cnn(df)

            #recording validation accuracy for different bp amounts
            print('Recording accuracy')
            for bp_valid in all_bp_tr:
                with learn.no_bar():
                    acc = get_accuracy(learn, df, [bp_valid])
                print(bp_valid, acc[1])
                
                entry = dict(kmer_size=kmer_len,
                             replicate = replicate,
                             bp_training = '|'.join([str(x) for x in sorted(all_bp_tr)]),
                             bp_valid = bp_valid,
                             samples_training = '|'.join(df.loc[~df['is_valid']]['sample'].drop_duplicates().sort_values().tolist()),
                             n_samp_training = df.loc[~df['is_valid']]['sample'].drop_duplicates().shape[0],
                             samples_valid = '|'.join(df.loc[df['is_valid']]['sample'].drop_duplicates().sort_values().tolist()),
                             n_samp_valid = df.loc[df['is_valid']]['sample'].drop_duplicates().shape[0],
                             valid_loss = acc[0],
                             valid_acc = acc[1])
                print(entry, file = outfile)
                
            
            #training with specific bp amounts
            for bp_train in all_bp_tr:
                clear_output()
                print('Training using', str(bp_train),'bp for kmer size', kmer_len)
                print('Replicate', replicate)
                
                df = get_training_set(kmer_len, [bp_train])
                learn = train_cnn(df)
                print('Recording accuracy')
                for bp_valid in all_bp_tr:
                    with learn.no_bar():
                        acc = get_accuracy(learn, df, [bp_valid])
                    print(bp_valid, acc[1])
                    entry = dict(kmer_size=kmer_len,
                                 replicate = replicate,
                                 bp_training = bp_train,
                                 bp_valid = bp_valid,
                                 samples_training = '|'.join(df.loc[~df['is_valid']]['sample'].drop_duplicates().sort_values().tolist()),
                                 n_samp_training = df.loc[~df['is_valid']]['sample'].drop_duplicates().shape[0],
                                 samples_valid = '|'.join(df.loc[df['is_valid']]['sample'].drop_duplicates().sort_values().tolist()),
                                 n_samp_valid = df.loc[df['is_valid']]['sample'].drop_duplicates().shape[0],
                                 valid_loss = acc[0],
                                 valid_acc = acc[1])
                    print(entry, file = outfile)

print('DONE')

Training using 200000000 bp for kmer size 9
Replicate 9


Recording accuracy
500000 0.20000000298023224
1000000 0.36666667461395264
2000000 0.46666666865348816
5000000 0.800000011920929
10000000 0.8333333134651184
20000000 0.8666666746139526
50000000 0.8999999761581421
100000000 0.8666666746139526
200000000 0.8666666746139526
DONE


Now that we finished training and testing all models, let's save results as a table that can be easily read in R:

In [6]:
with open('kmerSize_VS_bp.txt','r') as infile:
    df = pd.DataFrame([eval(x) for x in infile])
    df.to_csv('kmerSize_VS_bp.csv')

Path('kmerSize_VS_bp.txt').unlink()